In [35]:
import pandas as pd
import scipy.signal
from google.oauth2 import service_account
from gsheetsdb import connect

from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [36]:
import toml
secrets_file = ".streamlit/secrets.toml"
secrets = toml.load(secrets_file)

In [43]:




# Create a connection object.
credentials = service_account.Credentials.from_service_account_info(
    secrets["gcp_service_account"],
    scopes=[
        "https://www.googleapis.com/auth/spreadsheets",
    ],
)
conn = connect(credentials=credentials)

# Perform SQL query on the Google Sheet.
# Uses st.cache to only rerun when the query changes or after 10 min.

def run_query(query):
    rows = conn.execute(query, headers=1)
    rows = rows.fetchall()
    return rows

sheet_url = secrets["private_gsheets_url"]

rows = run_query(f'SELECT * FROM "{sheet_url}"')


def get_data():
    # TODO: must be a better way to get rows into dataframe.. see google docs
    df = pd.DataFrame({
        'date': [row.Date for row in rows],
        'distance': [row.Distance for row in rows],
        'speed': [row.Speed for row in rows],
        'avg_hr': [row.Avg_HR for row in rows]
    })
    df['duration (hr)'] = df['distance'] / df['speed']
    df.sort_values(by='date', inplace=True)
    
    df['hr_smooth'] = df['avg_hr'].rolling(6, win_type='cosine').mean()
    return df

df = get_data()

def make_distance_plot():
    p = figure(x_axis_type="datetime", title="Running Distances", plot_height=350, plot_width=800)
    p.background_fill_color = "#97ead2"
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Distance (mi)'

    p.line(df['date'], df['distance'], width=3, color="#FEA572")
    p.circle(df['date'], df['distance'], size=8, color="#FE640B")
    return p

distance_plot = make_distance_plot()




def make_hr_plot():
    p = figure(x_axis_type="datetime", title="Average HR", plot_height=350, plot_width=800)
    p.background_fill_color = "#97ead2"
    # p.xgrid.grid_line_color=None
    # p.ygrid.grid_line_alpha=0.8
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'HR (bpm)'

    p.line(df['date'], df['avg_hr'], width=3, color="#FEA572", legend_label="HR (bpm)")
    p.circle(df['date'], df['avg_hr'], size=8, color="#FE640B")
    
    p.line(df['date'], df['hr_smooth'], width=3, legend_label="HR smoothed")#color="#FEA572")
    p.legend.location = "bottom_left"
    return p

hr_plot = make_hr_plot()


show(hr_plot)

In [25]:
df

,date,distance,speed,avg_hr,duration (hr),hr_smooth
0,2022-05-16,4.66,6.81,155.7,0.684288,NaN
1,2022-05-21,7.01,5.43,145.4,1.290976,NaN
2,2022-05-25,5.05,6.04,143.0,0.836093,148.033333
3,2022-05-26,2.48,6.42,141.0,0.386293,143.133333
4,2022-05-28,2.95,5.72,136.1,0.515734,140.033333
5,2022-05-29,7.74,5.14,141.1,1.505837,139.400000
6,2022-05-31,3.03,6.09,150.6,0.497537,142.600000
7,2022-06-01,5.05,6.32,143.6,0.799051,145.100000
8,2022-06-02,3.12,6.44,139.6,0.484472,144.600000
9,2022-06-04,3.47,6.50,140.7,0.533846,141.300000
